<a href="https://colab.research.google.com/github/berthine/Deep_NLP/blob/master/Copy_of_ammi_dnlp_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chit Chat Chatbots

In the previous lab, we explored models that try to answer questions by reasoning over free-text input. In this lab, we will explore two types of models to create chatbots.

First, let's consider important qualities for a chit-chat chatbot system


1.   **Readability** - whatever model we use, the chats it creates should be easily understood by humans
2.   **Consistency** - when chatting with a chatbot, the bot should maintain consistent information. Imagine a bot that says "Hi I'm Jack'' and then "Hello, my name is Jane" - quite confusing
3.    **Engaging** - To encourage users to talk to the bot, the bot should be able to generate interesting, engaging responses. If the only response was "wow, that's cool," users are quite unlikely to want to talk very much to the chat bot



In [ ]:
%%html
<p style='color: blue;'>
  Throughout the lab, there will be <b>questions</b> you should answer. <b>All questions you need to write an answer to will be in this blue color.</b>
  
  <br>Please write brief answers- no need for long explanations. 
  <br>There can be multiple correct answers to the questions.
  
  <br><br>The goal of these questions is to:
  <ul style='color: green;'>
    <li> Review the lecture material in the context of practical models and develop intuition about the models
    <li> Develop a sense of experimentation - we will pretend we have a dataset and will walk through an experimental thought process.
  </ul>

<b>We are going to do the lab as a group. <br>I will explain the sections in more depth, as we did not cover dialogue deeply during the lecture. <br> After we discuss, I will provide time for you to write a few sentences. At the end of the lab, you will hand it in. In theory, everyone should be finished together!</b>

  
</p>

## Data

The dataset we will use for this lab is called `PersonaChat` - it was created to directly address problem 2. Each person talking in the dataset has a personality, which helps maintain consistency in the dialogue. We saw it last week in the tutorials as well (when you worked through beam search)

In [ ]:
!git clone https://github.com/facebookresearch/ParlAI.git ~/ParlAI  > /dev/null
!cd ~/ParlAI; python setup.py develop > /dev/null

Cloning into '/root/ParlAI'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 29746 (delta 47), reused 43 (delta 19), pack-reused 29654
Receiving objects: 100% (29746/29746), 57.90 MiB | 25.09 MiB/s, done.
Resolving deltas: 100% (21144/21144), done.
zip_safe flag not set; analyzing archive contents...
/usr/local/lib/python3.6/dist-packages/setuptools/dist.py:454: UserWarning: Normalizing '2019.08.19' to '2019.8.19'
  warnings.warn(tmpl.format(**locals()))
regex_3/_regex.c: In function ‘folded_char_at’:
regex_3/_regex.c:10625:9: warning: variable ‘folded_len’ set but not used [-Wunused-but-set-variable]
     int folded_len;
         ^~~~~~~~~~
regex_3/_regex.c: In function ‘fuzzy_match_group_fld’:
regex_3/_regex.c:11503:10: warning: ‘data.new_text_pos’ may be used uninitialized in this function [-Wmaybe-uninitialized]
     if (!record_fuzzy(state, data.fuzzy_type, data.new_text_pos - dat

**Example: **

your persona: i just started college.

your persona: i have 3 science classes.

your persona: i work part time in the campus library.

your persona: i am living at home but hope to live in the dorms next year.

**Partner Dialogue**: hi how are you doing

**Your Response**: great ! just got off work and relaxing before i study

In [ ]:
# let's download and take a look at some examples of data in PersonaChat
!python ~/ParlAI/examples/display_data.py --task personachat --datatype train

[creating task(s): personachat]
[building data: /root/ParlAI/data/Persona-Chat]
[ downloading: http://parl.ai/downloads/personachat/personachat.tgz to /root/ParlAI/data/Persona-Chat/personachat.tgz ]
[ Checksum Successful ]
unpacking personachat.tgz
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
- - - NEW EPISODE: personachat - - -
your persona: i like to remodel homes.
your persona: i like to go hunting.
your persona: i like to shoot a bow.
your persona: my favorite holiday is halloween.
hi , how are you doing ? i am getting ready to do some cheetah chasing to stay in shape .
   you must be very fast . hunting is one of my favorite hobbies .
i am ! for my hobby i like to do canning or some whittling .
   i also remodel homes when i am not out bow hunting .
that is neat . when i was in high school i placed 6th in 100m dash !
   that is awesome . do you have a favorite season or time of year ?
i do not . but i do have a favorite meat since tha

In [ ]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What do the personalities look like?</li>
    <li>How does creating bots with these simple personalities address consistency for chatbots? </li>
    <li>What are some drawbacks/limitations of these specific personalities for addressing the problem of consistency?</li>
  </ul>
</p>

**Answers**

**- What do the personalities look like?**

The personnalities are looking simple, interesting  and clear. 

 **- How does creating bots with these simple personalities address consistency for chatbots?**

 With these simple personalities the domain of conversation with  is limited. The chatbot can not go out. For example if the personalities are about footbal you can not ask a question related to finance.

 **- What are some drawbacks/limitations of these specific personalities for addressing the problem of consistency?**

The drawbacks/limitations of these specific personalities:

- We can not ask general questions, it only contains limited personalities.
- It too focus on the domain.




---


***Let's understand how much data we have. Let's compute the following using ParlAI:***


1.   **How many turns of data do we have?** In dialogue datasets, "amount of data" is measured in dialogue turns. Each time there is a single line of dialogue, that is called a "turn"
2.   **On average, how many words form a model input?**


---




In [ ]:
!python ~/ParlAI/parlai/scripts/data_stats.py -t personachat -dt train -ltim 10000

[ note: changing datatype from train to train:ordered ]
[creating task(s): personachat]
[loading fbdialog data:/root/ParlAI/data/Persona-Chat/personachat/train_self_original.txt]
[ loaded 8939 episodes with a total of 65719 examples ]
16s elapsed:
      %done  \
    100.00%   
      exs  \
    65719   
                                                                                                                                                                                                                                                                                                                                                                                                                                                                 stats  \
    
input:
   utterances: 65719
   avg utterance length: 18.356974390967604
   tokens: 1206402
   unique tokens: 14209
   unique utterances: 64580
labels:
   utterances: 65719
   avg utterance length: 11.929411585690591
   tokens: 783989

## Evaluation

How are dialogue models evaluated?



1.   **Automatic Evaluation**: Hits @ 1, Hits @ 5, Hits @ 10, F1
2.   **Human Evaluation**: Pairing Selection, Human Rating



In [ ]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Take some notes about what these metrics are and what they mean here</li>
  </ul>
</p>

**- Evaluation** is about to understand and know when your model is made  improvevement.

**- Automatic Evaluation** basically means run a script and get an answer immediatelly. 

**- Hits @k**: means that if you have a set of retrieval candidates for examples k= 100, for  my model to say hit @k means the rate of correct utterance appearing in the top k entries for each instance list. 
It's very useful for retrieval based dialogue models.

**- Hits @1** means that when the model has scored all the candidates and the highest ranked one is the correct utterance.

**- Hits @5** means that the correct answers are ranked in the top 5.

**- Hits @10** means that the correct answers are ranked in the top 10.

**- F1** : is a mix of precision and recall. It can be use to evaluate generative and retrieval models. It takes the human text and split into individual words and takes also what the model has written, plits into terms of words and calcutalte the precision and recall.

**- Human Evaluation** basically need a human to read and decide how good  is the model. 

**- Pairing selection** in pairing we compare two items.
**- Human ranking**  when humans are asked to rate the model based on several criteria such as task completion success, comprehension. etc.


## Models

There are two main kinds of dialogue models. 

**Retrieval** Models analyze the current dialogue context and try to find appropriate responses in the dataset.

**Generative** Models analyze the current dialogue context
and try to write an answer, word by word, from left to right.
This can be thought of as an application of sequence-to-sequence models,  where the "encoder side" is the dialogue history and the "decoder side" is the dialogue response your chatbot should generate.

In [ ]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Let's discuss the pros/cons of retrieval compared to generative models - Are there settings when you might want to use one over the other?</li>
    <li>Compare a chit-chat application to something like booking a movie ticket- would you want to use generative, retrieval, or something else to accomplish that task? Why?</li>
    <li>How can you evaluate generative models with the metrics we discussed before? How do you think they will perform compared to retrieval models?</li>
    <li>In lecture, Antoine mentioned issues with generative model generation being generic and short. How does this happen in beam search?</li>
</ul>

<b> If you would like me to discuss how to actually use generative models in dialog, please say something! Otherwise, we will skip.</b>
</p>

**Answers**

**- Let's discuss the pros/cons of retrieval compared to generative models**

**Retrieval models** the benefit is that if you retrieve you don't have to make  mistakes if your entire training set is basically built of things that human said. You won't make mistakes in grammar.

**Generative models**  can generate new text, they're  not as restricted to the training set if they completely have not seen terms of words. They're able to adapt to a new situation, the capacity to generate responses even  if they do not figure in the corpus. They are quite likely to make grammatical mistake especially for long sentences.

**- Are there settings when you might want to use one over the other?**

- Generative models require huge amount of training data otherwise use the retrieval models which doesn't require a huge training data..
- In Retrieval models the candidate responses should be limited if not the model will run forever. 

**- Compare a chit-chat application to something like booking a movie ticket- would you want to use generative, retrieval, or something else to accomplish that task? Why?**

Booking a movie ticket, the generative or retrieval models will no be able to accomplish the tasks. Because these  can not accomplish completly (100%)  the task, we need something else what  is specific, not expensive to accomplish  perfectly the task.

**- How can you evaluate generative models with the metrics we discussed before? How do you think they will perform compared to retrieval models?**

Hits @ is the evaluation metric used for retrieval models and F1 can be use for both retrieval and generative models.To perform  generally, it  depends on the specific quality of the model. For example you train the model using a basketball dataset and test using the soccer dataset,  the generative models will perform better than the retrieval models. 

**- In lecture, Antoine mentioned issues with generative model generation being generic and short. How does this happen in beam search?**

When generative models are put into situation where they don't know what to say, they often say something super generic and short for example "Hello! how are you?", "I don't know". 

In beam search, every step of the beam search, the model try to predict and depending of size of the beam probable next candidates and then it selects like the top performing beam size candidates to continue in the beam search.

Since beam search works by maximizing the probability over the sentence, if something new happen, it will be penalize by the beam search.


### Retrieval Models

Let's train a model to do retrieval first. We will try the *Memory Net.* 

In [ ]:
# We can train a model with the following command: 
# !python ~/ParlAI/examples/train_model.py -m kv_memnn -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy

# but we have limited time in the tutorial, so let's use an already pretrained model

Quick Parameter Refresher:


*  `-m ` means which model we're going to use. Recall retrieval models are trained to rank the true response higher over a set of potential responses from the dataset (in ParlAI, these are called the "label candidates"). When it's time to write a dialogue response, the retrieval model returns the response that is ranked the highest
*  -`t` refers to the task. Here, we are training on PersonaChat data.
* `-dt` refers to the data split. We want to train our model, so we are using the training set.
* `-veps` refers to how often we should evaluate during training, our performance on validation. recall this is important because models, particularly neural ones, have the capacity to memorize the training dataset. So it's important to check how the model is doing on the validation set.
* `--model-file` refers to when your model is saved, what should the filename be
*  `-vmt` refers to the metric which we'll use to decide which model is the best. We'll cover this in the next section





**Let's interact with the model to get a sense of what it's learning. **How is this chat going to work?



1.   You will be assigned a persona. You will chat to the model by typing in the chat box.
2.   The chatbot also has a persona. It's secret and hidden from you!
3.   When you've finished chatting with this bot, type [DONE] and a new model persona will be assigned to the bot, so you can talk to a new bot. 
4.   When you move on to the next chatbot persona, the previous persona will be revealed. 

Interact with the chatbots and the personas. **Try to think about the following:**

*   Do the chatbots follow their persona a lot?
*   Was it difficult to follow your persona?





In [ ]:
!python ~/ParlAI/projects/convai2/interactive.py -mf models:convai2/kvmemnn/model

[building data: /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz]
[ downloading: http://parl.ai/downloads/_models/convai2/kvmemnn.tgz to /root/ParlAI/data/models/convai2/kvmemnn/kvmemnn.tgz ]
unpacking kvmemnn.tgz
[ warning: overriding opt['model_file'] to /root/ParlAI/data/models/convai2/kvmemnn/model (previously: /checkpoint/jase/20180328/kvmemnn_sweep10/persona-self_rephraseTrn-True_rephraseTst-False_lr-0.1_esz-2000_margin-0.1_tfidf-False_shareEmb-True_hops1_lins0/model )]
[ creating KvmemnnAgent ]
Dictionary: loading dictionary from /root/ParlAI/data/models/convai2/kvmemnn/model.dict
[ num words =  19153 ]
Loading existing model params from /root/ParlAI/data/models/convai2/kvmemnn/model
[loading candidates: /root/ParlAI/data/models/convai2/kvmemnn/model.candspair*]
[caching..]
=init done=
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.form

In [ ]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>What does this model seem to be doing well? What is it doing poorly? </li>
    <li>Why might it be performing poorly? What kind of experiment could you design to test your hypothesis?</li>
    <li>How do we know if we need to use a more complex model? Would we always want to use a more complex model? Why or why not?</li>
  </ul>
</p>

**Answers**

**What does this model seem to be doing well? What is it doing poorly?** 

What the model does well is that when you type a message that the model knows is giving a good answer otherwise is trying to give an answer  even if it not related to what you are talking about.

What it doing poorly is that sometimes is giving wrong answer not related to your message.


**Why might it be performing poorly? What kind of experiment could you design to test your hypothesis?**

The A/B testing can be used to improve  a chatbot, to  optimize for better results.

So before I begin creating your chatbot experiments, I will first choose the metrics that I want to improve.

For example, if am using a chatbot for marketing, my metric could be the number of leads who opt-in after a successful chatbot interaction.

Alternatively, if am using a chatbot for boosting sales, my metric could be the number of trial leads whose engagement score improves because of interacting with the chatbot.

Whatever it is, once I’ve identified the metric  to optimize, I can start  working on the chatbot experiment.

A typical A/B test involves three steps:

  - The problem. The issue I want to solve
  - The control. The existing feature or experience I am testing against, without which, I wouldn’t know whether my experiment was successful

- The proposed solution. The new approach I’ve come up with to solve the problem my hypothesis.

**How do we know if we need to use a more complex model? Would we always want to use a more complex model? Why or why not?**

In [ ]:
# Here is a command to train a Transformer Ranker model if you would like to try it out
# !python ~/ParlAI/examples/train_model.py -m transformer/ranker -t personachat -dt train -veps 0.25 --model-file persona_chat_retrieval_model -vmt accuracy



---


An important aspect of training models is analyzing them. ***Try to answer the following questions.*** 

---




In [ ]:
%%html
<p style='color: blue;'>
  <b>Questions:</b>
  <ul style='color: blue;'>
    <li>Are the models using the persona that we have provided? How can you tell? If I asked you to prove it to me, what experiments could you conduct? </li>
    <li>Previously, we computed some statistics about how long the persona is in the training data. The model has also only seen words present in the training dataset. But what happens if you push the model outside of what data it's been trained on? What kind of performance do you get? Why does this happen, and what could you do if you wanted to improve the model's ability to generalize? </li>
    <li>In ParlAI, we've set the parameters to save the model's best performance based on validation accuracy. What would happen if we saved the model based on the best training accuracy? Why does this happen? (if you like, try this out on your own and see the effect when you interact with the bot)</li>
  </ul>

<b> If you would like me to discuss how to use BERT in dialog, please say something! Otherwise, we will skip.</b>
</p>

### [for self exploration] Generative Models

Generative models must produce word for word what they are going to say next in the dialogue. When predicting the next word, it produces a probability distribution over the entire vocabulary space for which word to generate next. To reduce the vocabulary space, we will use **byte-pair encoding** (BPE). 

*How does BPE work?* The BPE algorithm takes as input the training data and the number of *operations* it can do. It passes over the training set and tries to create sub-word units. For example, the word "beautiful" might be split into "beau" "ti" "ful". Each time it splits a word into sub-words, that is one operation. The final vocabulary output consists of these subwords. So "ful" can be part of "beautiful" and part of "fruitful" and so on.


**Questions to ask yourself**:


1.   Why is it important to keep the vocabulary space small?
2.   What does perplexity measure? Why would we use it as a training objective? 




In [ ]:
# !python ~/ParlAI/examples/train_model.py -m transformer/generator -t personachat -dt train -veps 0.25 --model-file persona_chat_generative_model -vmt ppl

## Final Thoughts

**What did we learn about dialogue modeling? Review Questions to ask yourself**

*   How do retrieval models work? What about generative? What are their pros and cons?
*   What are some important traits of dialogue systems? How might the traits differ for different dialogue tasks?


**General Takeaways about Machine Learning and Experimentation:**

*   We don't try models just to try them - try to have a reason for conducting an experiment. As we did in the lab, try to analyze what's working well in your models and working poorly. Try to use these reasons to guide why you might want to try other models. Complex is not necessarily better. 
*   Certain models can be better for certain tasks. As we've seen, generative models are working really well for tasks such as machine translation, but have a bit to go before becoming general purpose dialogue generators. 



**I'm really interested in dialogue! What can I do to learn more?**


*   Play around in ParlAI: ParlAI is a general library with many great dialogue models and code for them. It also provides a standard interface to access datasets and interact with various models. 
*   Read the PersonaChat Paper: https://arxiv.org/pdf/1801.07243.pdf
*   Dialog using knowledge: One challenge of these chit chat systems is they do not concretely know any facts. So if you want to chat about a specific topic, the models cannot produce any relevant information - they say generic utterances or incorrect facts. One way to remedy this is to incorporate **knowledge** into the dialogue agents. This has been investigated in many different ways, but one of the first papers to show this is https://arxiv.org/abs/1811.01241. In this work, data is collected by asking one speaker to reference Wikipedia sentences.
* Dialog with BERT: pretty new,  there is an investigation of two ways to use BERT in this paper: https://arxiv.org/abs/1903.03094. 


